In [1]:
import geopandas
import pandas as pd
import os
import rasterio
from rasterio.plot import show
from rasterio.mask import mask
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import rioxarray
import glob
import pandas as pd


In [2]:
data_folder="./processed_data"
data_files={
    "elevation":"processed_data\elevation_dataset\Elevation_DZ_TN.tif",
    "fire":r"processed_data\fire_dataset\fire.csv",
    "landcover":"processed_data\landcover_dataset\landcover_DZ_TN.shp",
  
}
output_folder="./nc_data"
output_folders={
    "elevation":"elevation_nc",
    "fire":"fire_nc",
    "landcover":"landcover_nc",
    "climate_prec":"climate_nc/prec_nc",
    "climate_tmax":"climate_nc/tmax_nc",
    "climate_tmin":"climate_nc/tmin_nc"

}



## LandCover

In [3]:
# Load datasets
landcover_merged = geopandas.read_file(data_files["landcover"])

In [4]:
landcover_merged["lon"] = landcover_merged.geometry.centroid.x
landcover_merged["lat"] = landcover_merged.geometry.centroid.y

C:\Users\BIG BOX\AppData\Local\Temp\ipykernel_22408\3242131276.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  landcover_merged["lon"] = landcover_merged.geometry.centroid.x
C:\Users\BIG BOX\AppData\Local\Temp\ipykernel_22408\3242131276.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  landcover_merged["lat"] = landcover_merged.geometry.centroid.y


In [6]:
import xarray as xr

# Select columns to include
ds = xr.Dataset({
    "LCCCODE": ("points", landcover_merged["LCCCODE"].astype(str)),
    "AREA": ("points", landcover_merged["AREA"]),
},
coords={
    "lon": ("points", landcover_merged["lon"]),
    "lat": ("points", landcover_merged["lat"])
})

In [8]:

output_base = os.path.join(output_folder, output_folders["landcover"])
os.makedirs(output_base, exist_ok=True)
output_nc = os.path.join(output_base, "landcover_DZ_TN.nc")
ds.to_netcdf(output_nc)
print(f"✅ Saved NetCDF file to: {output_nc}")
import netCDF4 as nc

dataset = nc.Dataset(output_nc)
print(dataset)


✅ Saved NetCDF file to: ./nc_data\landcover_nc\landcover_DZ_TN.nc
<class 'netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): points(438513)
    variables(dimensions): <class 'str'> LCCCODE(points), float64 AREA(points), float64 lon(points), float64 lat(points)
    groups: 


## Climate


In [3]:
def convert_climate_to_nc(files_path,name):
    tif_files = sorted(glob.glob(files_path))

    # Step 2 — Open and stack all TIFFs
    data_list = [rioxarray.open_rasterio(f).squeeze() for f in tif_files]

    # Step 3 — Combine into a single xarray Dataset
    combined = xr.concat(data_list, dim="time")
    combined.name = name
    return combined

In [4]:
prec_combined=convert_climate_to_nc(r'processed_data\climate_dataset\prec\*.tif',"precipitation")
tmax_combined=convert_climate_to_nc(r'processed_data\climate_dataset\tmax\*.tif',"tmax")
tmin_combined=convert_climate_to_nc(r'processed_data\climate_dataset\tmin\*.tif',"tmin")

In [5]:
output_base = os.path.join(output_folder, output_folders["climate_prec"])
os.makedirs(output_base, exist_ok=True)
output_nc = os.path.join(output_base, "precipitation_DZ_TN.nc")
prec_combined.to_netcdf(output_nc)

In [6]:
output_base = os.path.join(output_folder, output_folders["climate_tmax"])
os.makedirs(output_base, exist_ok=True)
output_nc = os.path.join(output_base, "tmax_DZ_TN.nc")
tmax_combined.to_netcdf(output_nc)

In [7]:
output_base = os.path.join(output_folder, output_folders["climate_tmin"])
os.makedirs(output_base, exist_ok=True)
output_nc = os.path.join(output_base, "tmin_DZ_TN.nc")
tmin_combined.to_netcdf(output_nc)

## Elevation

In [8]:
file_path=data_files["elevation"]
elevation_data = rioxarray.open_rasterio(file_path).squeeze()
output_base = os.path.join(output_folder, output_folders["elevation"])
os.makedirs(output_base, exist_ok=True)
output_nc = os.path.join(output_base, "elevation_DZ_TN.nc")
elevation_data.to_netcdf(output_nc)